In [160]:
import warnings
warnings.simplefilter('ignore')

# %matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from datetime import date, datetime
from sklearn.metrics import mean_squared_error
from sklearn import ensemble
from sklearn import datasets
from sklearn.utils import shuffle

In [161]:
# Read the csv files into a pandas DataFrame
train = pd.read_csv('master_train.csv', index_col = 0)
test = pd.read_csv('master_test.csv',index_col = 0)

In [162]:
train.head(2)

,id,belongs_to_collection_True,budget,language_enc,popularity,status_Released,year,month,Action,Adventure,...,count_crew,count_staff,count_keyword,castWt,prodcoWt,keywordWt,crewWt,teamWt,inEnglish,revenue
0,1,1,14000000,0,6.575393,1,2015,2,0,0,...,72,96,4,597,19,80,1930,2546,1,12314651
1,2,1,40000000,0,8.248895,1,2004,8,0,0,...,9,29,4,385,0,53,158,543,1,95149435


In [163]:
train.describe()

,id,belongs_to_collection_True,budget,language_enc,popularity,status_Released,year,month,Action,Adventure,...,count_crew,count_staff,count_keyword,castWt,prodcoWt,keywordWt,crewWt,teamWt,inEnglish,revenue
count,3000.000000,3000.000000,3.000000e+03,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,...,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3.000000e+03
mean,1500.500000,0.201333,2.253133e+07,1.328667,8.463274,0.998667,1999.713000,6.775333,0.247000,0.146333,...,24.365667,44.969333,7.226667,460.212667,35.439000,126.824000,568.334667,1063.986333,0.872667,6.672585e+07
std,866.169729,0.401063,3.702609e+07,4.179370,12.104000,0.036497,15.423313,3.409115,0.431339,0.353499,...,28.066419,37.700303,6.665891,422.121299,37.548372,125.654519,716.145497,959.777160,0.333402,1.375323e+08
min,1.000000,0.000000,0.000000e+00,0.000000,0.000001,0.000000,1921.000000,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000e+00
25%,750.750000,0.000000,0.000000e+00,0.000000,4.018053,1.000000,1993.000000,4.000000,0.000000,0.000000,...,8.000000,21.000000,3.000000,223.000000,9.000000,44.000000,151.000000,457.000000,1.000000,2.379808e+06
50%,1500.500000,0.000000,8.000000e+06,0.000000,7.374861,1.000000,2004.000000,7.000000,0.000000,0.000000,...,14.000000,33.000000,6.000000,339.000000,26.000000,99.000000,298.000000,739.500000,1.000000,1.680707e+07
75%,2250.250000,0.000000,2.900000e+07,0.000000,10.890983,1.000000,2011.000000,10.000000,0.000000,0.000000,...,28.000000,56.000000,10.000000,536.000000,49.000000,180.000000,643.000000,1297.250000,1.000000,6.891920e+07
max,3000.000000,1.000000,3.800000e+08,35.000000,294.337037,1.000000,2017.000000,12.000000,1.000000,1.000000,...,194.000000,280.000000,149.000000,3954.000000,370.000000,2782.000000,4660.000000,6946.000000,1.000000,1.519558e+09


In [164]:
test.head(2)

,id,belongs_to_collection_True,budget,language_enc,popularity,status_Released,year,month,Action,Adventure,...,count_cast,count_crew,count_staff,count_keyword,castWt,prodcoWt,keywordWt,crewWt,teamWt,inEnglish
0,3001,1,0,0,3.851534,1,2007.0,7.0,0,1,...,7,2,9,2,120,0,23,36,156,1
1,3002,0,88000,1,3.559789,1,1958.0,5.0,0,0,...,10,8,18,8,185,5,161,197,387,1


In [165]:
# Assign the data to X and y
Xid = train[['id']]
X_train = train[['belongs_to_collection_True', 'budget', 'popularity','year', 'month','count_cast', "castWt",'prodcoWt','keywordWt','crewWt','teamWt','count_genre','year','count_staff','count_keyword','count_crew']]
y_train = train['revenue'].values.reshape(-1,1)
print(X_train.shape, y_train.shape)


(3000, 16) (3000, 1)


In [166]:
testID = test['id']
testID.head()
test = test[['belongs_to_collection_True', 'budget', 'popularity','year', 'month','count_cast', "castWt",'prodcoWt','keywordWt','crewWt','teamWt','count_genre','year','count_staff','count_keyword','count_crew']]
print(test.shape)

(4398, 16)


In [167]:
feature_names = X_train.columns
feature_names

Index(['belongs_to_collection_True', 'budget', 'popularity', 'year', 'month',
       'count_cast', 'castWt', 'prodcoWt', 'keywordWt', 'crewWt', 'teamWt',
       'count_genre', 'year', 'count_staff', 'count_keyword', 'count_crew'],
      dtype='object')

### Standard Scaler

In [168]:
from sklearn.preprocessing import StandardScaler
X_scaler = StandardScaler().fit(X_train)
Y_scaler = StandardScaler().fit(y_train)
test_scaler = StandardScaler().fit(test)

X_train = X_scaler.transform(X_train)
test = X_scaler.transform(test)
y_train = y_scaler.transform(y_train)

## GradientBoostingRegressor -Standard Scaler

class sklearn.ensemble.GradientBoostingRegressor(loss=’ls’, learning_rate=0.1, n_estimators=100, subsample=1.0, criterion=’friedman_mse’, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_depth=3, min_impurity_decrease=0.0, min_impurity_split=None, init=None, random_state=None, max_features=None, alpha=0.9, verbose=0, max_leaf_nodes=None, warm_start=False, presort=’auto’, validation_fraction=0.1, n_iter_no_change=None, tol=0.0001)[source]¶

In [169]:
# Fit regression model
params = {'n_estimators': 500, 'max_depth': 4, 'min_samples_split': 2,
          'learning_rate': .01, 'loss': 'ls'} 
gbr = ensemble.GradientBoostingRegressor(**params)
predictions2 = gbr.fit(X_train,y_train)
training_score = gbr.score(X_train, y_train)
print(f"Training Score: {training_score}")

Training Score: 0.8635996676991662


In [170]:
sorted(zip(gbr.feature_importances_, feature_names), reverse=True)

[(0.6370055860496686, 'budget'),
 (0.15539888775769736, 'popularity'),
 (0.05589221579397939, 'belongs_to_collection_True'),
 (0.03695353958450558, 'count_cast'),
 (0.023005107354374287, 'month'),
 (0.018294947861564218, 'prodcoWt'),
 (0.010613939533026728, 'count_keyword'),
 (0.010382128573771398, 'keywordWt'),
 (0.00835507935438742, 'teamWt'),
 (0.007563717406239029, 'count_crew'),
 (0.007298487070794951, 'count_staff'),
 (0.006708730757940374, 'castWt'),
 (0.0066313439761257935, 'year'),
 (0.005888125253796495, 'count_genre'),
 (0.005481430209448372, 'year'),
 (0.00452673346268003, 'crewWt')]

In [171]:
#Make New Predictions
gbr_predictions = pd.DataFrame(y_scaler.inverse_transform(gbr.predict(test)), columns = ['revenue'])
output = pd.concat([testID, gbr_predictions], axis=1)
output.head()

,id,revenue
0,3001,1.888491e+07
1,3002,4.689463e+06
2,3003,1.443568e+07
3,3004,3.282832e+07
4,3005,6.199586e+06


In [172]:
output.to_csv("outputs/output_" + str(date.today()) + "_16" + ".csv", index=False)